In [1]:
def rank(r):
    return r.split("\\")[1].split("-")[0]

In [2]:
import pandas as pd
import glob
csv_f = pd.Series(glob.glob("./YT/*.csv"))
df = pd.DataFrame({"rank":csv_f.apply(rank), "path":csv_f})
df1 = df[[int(r) // 10 == 0 for r in df["rank"]]]
df2 = df.drop(df1.index)
df = pd.concat([df1, df2]).reset_index(drop=True)
df

,rank,path
0,1,./YT\1-蔡阿嘎.csv
1,2,./YT\2-這群人 TGOP.csv
2,3,./YT\3-館長.csv
3,4,./YT\4-王宏哲教養、育兒寶典.csv
4,10,./YT\10-眾量級CROWD.csv


In [3]:
import os

name_list = []
video_num_list = []

for f in df["path"]:
    df3 = pd.read_csv(f)
#     name = f.split("\\")[1].split("-")[1].split(".")[0]
    name = os.path.splitext(f)[0].split("\\")[1].split("-")[1]
    name_list.append(name)
    video_num = df3.shape[0]
    video_num_list.append(video_num)
df["name"] = name_list
df["video_num"] = video_num_list
df

,rank,path,name,video_num
0,1,./YT\1-蔡阿嘎.csv,蔡阿嘎,644
1,2,./YT\2-這群人 TGOP.csv,這群人 TGOP,201
2,3,./YT\3-館長.csv,館長,1227
3,4,./YT\4-王宏哲教養、育兒寶典.csv,王宏哲教養、育兒寶典,288
4,10,./YT\10-眾量級CROWD.csv,眾量級CROWD,436


In [4]:
df_all = pd.read_csv("ALL.csv")
df_class = df_all[["名次", "分類"]]
df["分類"] = df_class[[str(r) in list(df["rank"]) for r in df_class["名次"]]].reset_index(drop=True)["分類"]
df = df.drop(["path"], axis=1)
df = df[["分類", "rank", "name", "video_num"]]
df
# df.to_csv("IG_num.csv", index=False)

,分類,rank,name,video_num
0,生活,1,蔡阿嘎,644
1,搞笑,2,這群人 TGOP,201
2,生活,3,館長,1227
3,親子,4,王宏哲教養、育兒寶典,288
4,生活,10,眾量級CROWD,436


In [5]:
classes = list(set(df["分類"]))
num_list = []
image_num_list = []
video_num_list = []
for c in classes:
    df_sub = df[[str(r) in c for r in df["分類"]]].reset_index(drop=True)
    num_list.append(df_sub.shape[0])
    video_num_list.append(df_sub["video_num"].sum())
df_final = pd.DataFrame([classes, num_list, video_num_list]).T
df_final.columns = ["分類", "人數", "影片數量"]
df_final = df_final.sort_values(by=["影片數量"], ascending=False).reset_index(drop=True)
df_final

# df_final.to_csv("IG_num_class.csv")

,分類,人數,影片數量
0,生活,3,2307
1,親子,1,288
2,搞笑,1,201


In [6]:
classes = list(df_final["分類"])
df_sub_all = pd.DataFrame()
for c in classes:
    df_sub = df[[str(r) in c for r in df["分類"]]].reset_index(drop=True)
    df_sub["video_num_percent"] = round(df_sub["video_num"] / df_sub["video_num"].sum() * 100, 2)
    df_sub_all = pd.concat([df_sub_all, df_sub])
pd.set_option('display.max_rows', 100)
df_sub_all = df_sub_all[["分類", "rank", "name", "video_num", "video_num_percent"]].reset_index(drop=True)
df_sub_all

# df_sub_all.to_csv("IG_num_percent.csv")

,分類,rank,name,video_num,video_num_percent
0,生活,1,蔡阿嘎,644,27.92
1,生活,3,館長,1227,53.19
2,生活,10,眾量級CROWD,436,18.90
3,親子,4,王宏哲教養、育兒寶典,288,100.00
4,搞笑,2,這群人 TGOP,201,100.00


生活 26 19794 5751
插畫 18 11088 3092
美妝 4 6901 3428
搞笑 9 6663 3925
親子 4 5958 2818
寵物 4 3171 755
音樂 3 1827 708